# (Draft) Data Quality in Databricks Workflows with Pydantic cntd.

> ⚠️ This is a draft of the agenda of the future meetup

Thursday, February 20, 2025


➡️ [Meetup Announcement](https://www.meetup.com/warsaw-data-engineering/events/305995327/) (FIXME)

The future meetup is a continuation of the following...

Zakładamy, że mamy 2 projekty. Pierwszy projekt z pydantic (libka w Pythonie), a drugi to "hello world" Databricks Asset Bundle project z przykładowym job'em. Nic specjalnie wyrafinowanego. Od tego zaczniemy.

Agenda:

1. 5 minut rozgrzewki na luźne pomysły na ten i przyszłe meetupy
    * News (new versions, new features, etc.)
1. 50 minut Live coding session, a w nim:
    * Za pomocą Databricks Asset Bundles (DAB), uruchomisz Databricks job z notebookiem z libką w Pythonie z Pydantic (takie tam "hello world"). Wszystko z pomocą uv do zarządzania projektem.
    * Stworzymy UDFa do walidacji rekordów, którego "uzbroimy" w pydantic'a. To główny cel meetupu, którego osiągnięcie będzie naszym "najosobityczniejszym" sukcesem 🥂
    * Może coś jeszcze, ale nie zdradzę teraz 🤷‍♂️
1. 5 minut Q&A / Zbieranie pomysłów na kolejne edycje


## Event Question

O czym chciał(a)byś usłyszeć podczas meetupu? Rzuć ciekawym pomysłem na kolejne edycje 🙏

1. staram się nadarzyć za tym co Jacek mówi i czegoś się dowiedzieć
1. framework w Python - best practices
1. DAB
1. continue exploring quality with dqx or dlt publish to different schemas as good standard for medalion
1. Plan rozwoju, doświadczenia zawodowe wymiataczy technologicznych
1. Jakieś zaawansowane data quality w DBR; może jakaś analiza wykorzystania narzędzi typu Polars/DuckDB dla jedno-node’owych klastrów?
1. Jeżeli uda mi się dołączyć, to będzie fajnie posłuchać dalszej cześci poprzedniego meetup'u :)
1. Podłączenie Master data w transformacjach Databricks
1. Pydantic
1. everything about databricks
1. Chcę rozwijać swoje umiejętności w databricks a ta seria spotkań to coś czego szukałem

# 📢 News (FIXME)


## New Versions

* Databricks CLI
* [uv 0.5.29](https://github.com/astral-sh/uv/releases/tag/0.5.29)
* [awscli 2.24.0](https://github.com/aws/aws-cli/releases/tag/2.24.0)


# Krótko o uv: `uv init`


The very recent change was to add `--bare` option to `uv init`. Why is this important?

<br>

```py
uv init --bare
```


* [Working on projects](https://docs.astral.sh/uv/guides/projects/)
* [Creating projects](https://docs.astral.sh/uv/concepts/projects/init/)
* [uv init](https://docs.astral.sh/uv/reference/cli/#uv-init)


# Live Coding Session